In [1]:
import keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler ,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import os
import numpy as np

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf

In [0]:
# training parameters
epochs = 100
# network parameters
num_classes = 10
num_dense_blocks = 3
growth_rate = 12
depth = 100
num_bottleneck_layers = (depth - 4) // (2 * num_dense_blocks)

num_filters_bef_dense_block = 2 * growth_rate
compression_factor = 0.5


In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
# input image dimensions
input_shape = X_train.shape[1:]
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=10/50, random_state=42)

num_classes = 10
# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 
y_cv = tf.keras.utils.to_categorical(y_cv, num_classes)

# convert from integers to floats
train_norm = X_train.astype('float32')
test_norm = X_test.astype('float32')
cv_norm = X_cv.astype('float32')
# normalize to range 0-1
X_train = train_norm / 255.0
X_test = test_norm / 255.0
X_cv = cv_norm /255.0	


170500096/170498071 [==============================] - 4s 0us/step


In [5]:
X_train.shape , X_cv.shape

((40000, 32, 32, 3), (10000, 32, 32, 3))

In [6]:
X_test.shape

(10000, 32, 32, 3)

In [7]:
 # start model definition
# densenet CNNs (composite function) are made of BN-ReLU-Conv2D

inputs = Input(shape=input_shape)
x = BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(num_filters_bef_dense_block,
           kernel_size=3,
           padding='same',
           kernel_initializer='he_normal')(x)
x = concatenate([inputs, x])

# stack of dense blocks bridged by transition layers
for i in range(num_dense_blocks):
    # a dense block is a stack of bottleneck layers
    for j in range(num_bottleneck_layers):
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(4 * growth_rate,
                   kernel_size=1,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(growth_rate,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        x = concatenate([x, y])

    # no transition layer after the last dense block
    if i == num_dense_blocks - 1:
        continue

    # transition layer compresses num of feature maps and reduces the size by 2
    num_filters_bef_dense_block += num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block * compression_factor)
    y = BatchNormalization()(x)
    y = Conv2D(num_filters_bef_dense_block,
               kernel_size=1,
               padding='same',
               kernel_initializer='he_normal')(y)
    x = AveragePooling2D()(y)


# add classifier on top
# after average pooling, size of feature map is 1 x 1
x = AveragePooling2D(pool_size=8)(x)
x= Conv2D(num_classes,kernel_size=1,padding='valid',
                kernel_initializer='he_normal',
                activation='softmax')(x)
outputs = Flatten()(x)    
#outputs = Dense(num_classes,kernel_initializer='he_normal',activation='softmax')(x)         

# instantiate and compile model
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 3)    0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 24)   672         activation[0][0]                 
_____________

In [0]:
from time import time
from datetime import datetime


from tensorflow.python.keras.callbacks import TensorBoard

filepath = "weights_best.hdf5"
history = tf.keras.callbacks.History()

# tensorboard
tensorboard = TensorBoard(log_dir="model_logs/{}".format(time()))

learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.3, 
                                            min_lr=0.5e-6)
checkpoint_save = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto', baseline=None, restore_best_weights=False)

callbacks_list = [checkpoint_save,learning_rate_reduction,history,tensorboard]

In [0]:
from keras.preprocessing.image import ImageDataGenerator
# create data generator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(X_train)       

In [10]:
# fit model
batch_size = 32
steps = X_train.shape[0]//batch_size


history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=steps, epochs=epochs, validation_data=(X_cv,y_cv), verbose=1, callbacks=callbacks_list)

Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
10000/1250 [================================================================================================================================================================================================================================================] - 10s 1ms/sample - loss: 1.2766 - acc: 0.4877

Epoch 00001: val_acc improved from -inf to 0.48770, saving model to weights_best.hdf5
1250/1250 [==============================] - 216s 173ms/step - loss: 1.5884 - acc: 0.4242 - val_loss: 1.5196 - val_acc: 0.4877
Epoch 2/100
10000/1250 [================================================================================================================================================================================================================================================] - 8s 811us/sample - loss: 1.7057 - acc: 0.5606

Epoch 00002: val_acc improved from 0.48770 to 0.56060, saving mode

In [11]:
# Test the model
model.load_weights('weights_best.hdf5')
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 7s 681us/sample - loss: 0.3245 - acc: 0.9127
Test loss: 0.3244841086283326
Test accuracy: 0.9127





*   At epoch 93 I recieved the validation accuracy of 91.93 with the optimizer Adam which works fine. I built the model on keras framework with data augmentation.
*   The test accuracy is 91.27% on 10,000 datapoints with 0.331 Test loss.



